### Import packages correction

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import tensorflow as tf

# Get your datasets

In [2]:
from data.dataset import H5PY_RW
from data.data_tools import read_metadata, males_keys, females_keys

H5_dic = read_metadata()
chunk_size = 512*10

males = H5PY_RW('test_raw_16k.h5py', subset = males_keys(H5_dic)).set_chunk(chunk_size).shuffle()
fem = H5PY_RW('test_raw_16k.h5py', subset = females_keys(H5_dic)).set_chunk(chunk_size).shuffle()
print 'Data with', len(H5_dic), 'male and female speakers'


Data with 40 male and female speakers


# Mixing the dataset

In [3]:
from data.dataset import Mixer

mixed_data = Mixer([males, fem], with_mask=False, with_inputs=True)

# Training set selection
mixed_data.select_split(0)

# Model pretrained loading

In [4]:
N = 256
max_pool = 128
batch_size = 8
learning_rate = 0.001

config_model = {}
config_model["type"] = "pretraining"

config_model["batch_size"] = batch_size
config_model["chunk_size"] = chunk_size

config_model["N"] = N
config_model["maxpool"] = max_pool
config_model["window"] = 1024

config_model["smooth_size"] = 20

config_model["alpha"] = learning_rate
config_model["reg"] = 1e-4
config_model["beta"] = 0.1
config_model["rho"] = 0.01

idd = ''.join('-{}={}-'.format(key, val) for key, val in sorted(config_model.items()))
config_model["type"] = "DPCL_train_front"

In [5]:
from models.adapt import Adapt
import config

full_id = 'soft-base-9900'+idd

folder='DPCL_train_front'
model = Adapt(config_model=config_model,pretraining=False)
model.create_saver()

path = os.path.join(config.workdir, 'floydhub_model', "pretraining")
# path = os.path.join(config.log_dir, "pretraining")
model.restore_model(path, full_id)

ID : AdaptiveNet-wispy-forest-8617
INFO:tensorflow:Restoring parameters from /home/anthony/das/floydhub_model/pretraining/AdaptiveNet-soft-base-9900-N=256--alpha=0.001--batch_size=8--beta=0.1--chunk_size=5120--maxpool=128--reg=0.0001--rho=0.01--smooth_size=20--type=pretraining--window=1024-/model.ckpt


## Connect DPCL model to the front end

In [6]:
from models.dpcl import DPCL

with model.graph.as_default():
    model.connect_front(DPCL)
    model.sepNet.output = model.sepNet.prediction
    model.cost = model.sepNet.cost
    model.freeze_front()
    model.optimize
    model.tensorboard_init()

Tensor("prediction/l2_normalize:0", shape=(?, ?, 256, 40), dtype=float32)
[<tf.Variable 'prediction/W:0' shape=(1, 600, 10240) dtype=float32_ref>, <tf.Variable 'prediction/b:0' shape=(10240,) dtype=float32_ref>, <tf.Variable 'prediction/forward_BLSTM_1/rnn/basic_lstm_cell/kernel:0' shape=(556, 1200) dtype=float32_ref>, <tf.Variable 'prediction/forward_BLSTM_1/rnn/basic_lstm_cell/bias:0' shape=(1200,) dtype=float32_ref>, <tf.Variable 'prediction/backward_BLSTM_1/rnn/basic_lstm_cell/kernel:0' shape=(556, 1200) dtype=float32_ref>, <tf.Variable 'prediction/backward_BLSTM_1/rnn/basic_lstm_cell/bias:0' shape=(1200,) dtype=float32_ref>, <tf.Variable 'prediction/forward_BLSTM_2/rnn/basic_lstm_cell/kernel:0' shape=(900, 1200) dtype=float32_ref>, <tf.Variable 'prediction/forward_BLSTM_2/rnn/basic_lstm_cell/bias:0' shape=(1200,) dtype=float32_ref>, <tf.Variable 'prediction/backward_BLSTM_2/rnn/basic_lstm_cell/kernel:0' shape=(900, 1200) dtype=float32_ref>, <tf.Variable 'prediction/backward_BLSTM_

In [7]:
from itertools import compress
with model.graph.as_default():
    global_vars = tf.global_variables()
    is_not_initialized = model.sess.run([~(tf.is_variable_initialized(var)) \
                                   for var in global_vars])
    not_initialized_vars = list(compress(global_vars, is_not_initialized))
    print not_initialized_vars
    if len(not_initialized_vars):
        init = tf.variables_initializer(not_initialized_vars)


[<tf.Variable 'prediction/W:0' shape=(1, 600, 10240) dtype=float32_ref>, <tf.Variable 'prediction/b:0' shape=(10240,) dtype=float32_ref>, <tf.Variable 'prediction/forward_BLSTM_1/rnn/basic_lstm_cell/kernel:0' shape=(556, 1200) dtype=float32_ref>, <tf.Variable 'prediction/forward_BLSTM_1/rnn/basic_lstm_cell/bias:0' shape=(1200,) dtype=float32_ref>, <tf.Variable 'prediction/backward_BLSTM_1/rnn/basic_lstm_cell/kernel:0' shape=(556, 1200) dtype=float32_ref>, <tf.Variable 'prediction/backward_BLSTM_1/rnn/basic_lstm_cell/bias:0' shape=(1200,) dtype=float32_ref>, <tf.Variable 'prediction/forward_BLSTM_2/rnn/basic_lstm_cell/kernel:0' shape=(900, 1200) dtype=float32_ref>, <tf.Variable 'prediction/forward_BLSTM_2/rnn/basic_lstm_cell/bias:0' shape=(1200,) dtype=float32_ref>, <tf.Variable 'prediction/backward_BLSTM_2/rnn/basic_lstm_cell/kernel:0' shape=(900, 1200) dtype=float32_ref>, <tf.Variable 'prediction/backward_BLSTM_2/rnn/basic_lstm_cell/bias:0' shape=(1200,) dtype=float32_ref>, <tf.Variab

# Model creation

# Pretraining the model 

In [8]:
nb_iterations = 1000

#initialize the model
model.sess.run(init)

for i in range(nb_iterations):
    X_in, X_mix, Ind = mixed_data.get_batch(batch_size)
    c = model.train(X_mix, X_in, learning_rate, i)
    print 'Step #'  ,i,' loss=', c 

    if i%20 == 0: #cost_valid < cost_valid_min:
        print 'DAS model saved at iteration number ', i,' with cost = ', c 
        model.save(i)

Step # 0  loss= 110.018
DAS model saved at iteration number  0  with cost =  110.018
Step # 1  loss= 103.141
Step # 2  loss= 97.1314
Step # 3  loss= 93.5372
Step # 4  loss= 89.62
Step # 5  loss= 87.0062
Step # 6  loss= 85.6691
Step # 7  loss= 83.2059
Step # 8  loss= 81.6887
Step # 9  loss= 79.4571
Step # 10  loss= 77.8962
Step # 11  loss= 77.3552
Step # 12  loss= 76.932
Step # 13  loss= 75.8065
Step # 14  loss= 75.296
Step # 15  loss= 75.0277
Step # 16  loss= 74.7348
Step # 17  loss= 73.1299
Step # 18  loss= 73.4137
Step # 19  loss= 73.7363
Step # 20  loss= 73.0521
DAS model saved at iteration number  20  with cost =  73.0521
Step # 21  loss= 72.4501
Step # 22  loss= 72.8737
Step # 23  loss= 72.354
Step # 24  loss= 71.8575
Step # 25  loss= 71.7113
Step # 26  loss= 72.0687
Step # 27  loss= 72.5334
Step # 28  loss= 72.239
Step # 29  loss= 71.8977
Step # 30  loss= 71.5458
Step # 31  loss= 71.2849
Step # 32  loss= 70.973
Step # 33  loss= 71.0748
Step # 34  loss= 71.0125
Step # 35  loss= 70

Step # 285  loss= 36.0191
Step # 286  loss= 34.7683
Step # 287  loss= 35.7239
Step # 288  loss= 35.5106
Step # 289  loss= 34.5272
Step # 290  loss= 34.627
Step # 291  loss= 35.0789
Step # 292  loss= 34.3879
Step # 293  loss= 33.8159
Step # 294  loss= 34.2258
Step # 295  loss= 33.4258
Step # 296  loss= 34.4677
Step # 297  loss= 33.5358
Step # 298  loss= 34.243
Step # 299  loss= 33.616
Step # 300  loss= 33.5213
DAS model saved at iteration number  300  with cost =  33.5213
Step # 301  loss= 33.7629
Step # 302  loss= 33.2803
Step # 303  loss= 33.4837
Step # 304  loss= 33.2983
Step # 305  loss= 33.4349
Step # 306  loss= 33.7973
Step # 307  loss= 33.8824
Step # 308  loss= 32.5734
Step # 309  loss= 32.7787
Step # 310  loss= 32.8504
Step # 311  loss= 32.5141
Step # 312  loss= 31.9862
Step # 313  loss= 32.5661
Step # 314  loss= 31.1083
Step # 315  loss= 31.8342
Step # 316  loss= 33.198
Step # 317  loss= 32.5348
Step # 318  loss= 32.259
Step # 319  loss= 31.5473
Step # 320  loss= 32.8877
DAS mo

Step # 568  loss= 29.4241
Step # 569  loss= 26.958
Step # 570  loss= 29.6225
Step # 571  loss= 28.8206
Step # 572  loss= 26.6519
Step # 573  loss= 27.454
Step # 574  loss= 27.4336
Step # 575  loss= 28.6741
Step # 576  loss= 28.7398
Step # 577  loss= 28.0965
Step # 578  loss= 25.0969
Step # 579  loss= 27.1277
Step # 580  loss= 26.4178
DAS model saved at iteration number  580  with cost =  26.4178
Step # 581  loss= 28.0311
Step # 582  loss= 27.8594
Step # 583  loss= 24.9138
Step # 584  loss= 27.6105
Step # 585  loss= 27.1619
Step # 586  loss= 24.9944
Step # 587  loss= 27.5943
Step # 588  loss= 27.862
Step # 589  loss= 27.998
Step # 590  loss= 29.102
Step # 591  loss= 29.4088
Step # 592  loss= 28.8716
Step # 593  loss= 27.879
Step # 594  loss= 27.8882
Step # 595  loss= 28.2256
Step # 596  loss= 27.4835
Step # 597  loss= 25.8809
Step # 598  loss= 29.8995
Step # 599  loss= 28.0197
Step # 600  loss= 28.499
DAS model saved at iteration number  600  with cost =  28.499
Step # 601  loss= 26.197

KeyboardInterrupt: 